# Part A Task 1

The model chosen is embedding model

The example provided below is one of the embedded field obtained from the assignment dataset and modified and merged into the embedded data model.

{'GHI_w/m2': 180.0,
 'Hotspot': [{'confidence': 74.0,
              'datetime': '2021-03-26T04:47:20',
              'latitude': -37.665,
              'longitude': 142.1339,
              'surface_temperature_celcius': 48.0},
             {'confidence': 74.0,
              'datetime': '2021-03-26T04:43:40',
              'latitude': -38.0564,
              'longitude': 142.8525,
              'surface_temperature_celcius': 48.0}],
 '_id': ObjectId('627bdca917bef61a267db5f0'),
 'air_temperature_celcius': 23.0,
 'date': '2021-03-26',
 'max_wind_speed': 26.0,
 'precipitation ': ' 0.00I',
 'precipitation_amount': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 60.6,
 'station': '948701',
 'windspeed_knots': 10.1}

The reason of choosing embedded data model is to have a better readability as the climate data may contain more than 1 or even more than 10 hotspot data. Hence, with the use of embedded data model, we can have a better readability to observe the embedded hotspot data even the number of datas embedded is large.

# Part A Task 2

In [1]:
import pymongo
from pymongo import MongoClient

import csv
import pandas as pd
import json

import pprint

In [2]:
client = MongoClient ()
db = client.fit3182_assignment_db
hotspot_hist = db.hotspot_hist
climate_hist = db.climate_hist
merged_hist = db.merged_hist

In [3]:
hotspot_hist.delete_many({})
climate_hist.delete_many({})
merged_hist.delete_many({})

Task 2 Q(1)

In [4]:
data_hotspot_hist = []
with open("FIT3182_Assignment/hotspot_historic.csv", 'r') as file:
    csvreader = csv.reader(file)
    header_hotspot_hist = next(csvreader)
    for row in csvreader:
        data_hotspot_hist.append(row)

In [5]:
data_climate_hist = []
with open("FIT3182_Assignment/climate_historic.csv", 'r') as file:
    csvreader = csv.reader(file)
    header_climate_hist = next(csvreader)
    for row in csvreader:
        data_climate_hist.append(row)

In [6]:
data_hotspot_hist = pd.DataFrame(data_hotspot_hist)
data_hotspot_hist.columns = header_hotspot_hist

data_climate_hist = pd.DataFrame(data_climate_hist)
data_climate_hist.columns = header_climate_hist


data_climate_hist["date"]=pd.to_datetime(data_climate_hist["date"])
data_climate_hist["precipitation_type"] = data_climate_hist["precipitation "].str.extract('([A-Z]+)')
data_climate_hist["precipitation_amount"] = data_climate_hist["precipitation "].str.replace('([A-Z]+)','')
data_climate_hist[["air_temperature_celcius","relative_humidity","windspeed_knots","max_wind_speed","GHI_w/m2","precipitation_amount"]]=data_climate_hist[["air_temperature_celcius","relative_humidity","windspeed_knots","max_wind_speed","GHI_w/m2","precipitation_amount"]].astype(float)

data_hotspot_hist[["latitude","longitude","confidence","surface_temperature_celcius"]]=data_hotspot_hist[["latitude","longitude","confidence","surface_temperature_celcius"]].astype(float)
data_hotspot_hist["date"]=pd.to_datetime(data_hotspot_hist["date"])

In [7]:
grouped = ( data_hotspot_hist.groupby(['date'])
              .apply(lambda x: x[['latitude','longitude','datetime', 'confidence','surface_temperature_celcius']].to_dict('r'))
              .reset_index()
              .rename(columns={0:'Hotspot'}))

df = data_climate_hist.merge(grouped, on = 'date', how = 'left')

df["date"]=df["date"].astype(str)

merged_data = df.to_dict(orient='records')
merged_hist.insert_many(merged_data)

In [8]:
# Task 2 Q(2)(A)

result = merged_hist.find({'date': '2021-12-12'})

for x in result:
    pprint.pprint(x)

{'GHI_w/m2': 156.0,
 'Hotspot': [{'confidence': 53.0,
              'datetime': '2021-12-12T00:45:38',
              'latitude': -37.903,
              'longitude': 145.25,
              'surface_temperature_celcius': 44.0}],
 '_id': ObjectId('62877ec8313b28f84f667c4c'),
 'air_temperature_celcius': 19.0,
 'date': '2021-12-12',
 'max_wind_speed': 12.0,
 'precipitation ': ' 0.00I',
 'precipitation_amount': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 55.3,
 'station': '948702',
 'windspeed_knots': 6.2}


In [9]:
# Task 2 Q(2)(B)

pprint.pprint(list(merged_hist.aggregate([
    { '$unwind' : '$Hotspot' },
    {'$match' :{ '$and' : [ { 'Hotspot.surface_temperature_celcius' : { '$gte': 65}}, { 'Hotspot.surface_temperature_celcius' : { '$lte': 100}}]} },
    {'$project' : {'Hotspot.latitude' : 1, 'Hotspot.longitude' : 1 , 'Hotspot.surface_temperature_celcius' : 1, 'Hotspot.confidence' : 1, '_id':0}}

     ])))


[{'Hotspot': {'confidence': 94.0,
              'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celcius': 73.0}},
 {'Hotspot': {'confidence': 97.0,
              'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celcius': 80.0}},
 {'Hotspot': {'confidence': 84.0,
              'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celcius': 71.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celcius': 99.0}},
 {'Hotspot': {'confidence': 80.0,
              'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celcius': 68.0}},
 {'Hotspot': {'confidence': 85.0,
              'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celcius': 98.0}},
 {'Hotspot': {'confidence': 90.0,
      

              'latitude': -36.7165,
              'longitude': 142.6155,
              'surface_temperature_celcius': 66.0}},
 {'Hotspot': {'confidence': 94.0,
              'latitude': -37.5812,
              'longitude': 142.7374,
              'surface_temperature_celcius': 75.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.5949,
              'longitude': 142.6857,
              'surface_temperature_celcius': 98.0}},
 {'Hotspot': {'confidence': 95.0,
              'latitude': -37.8247,
              'longitude': 143.6243,
              'surface_temperature_celcius': 77.0}},
 {'Hotspot': {'confidence': 95.0,
              'latitude': -37.4028,
              'longitude': 143.6932,
              'surface_temperature_celcius': 75.0}},
 {'Hotspot': {'confidence': 92.0,
              'latitude': -37.4033,
              'longitude': 143.6884,
              'surface_temperature_celcius': 70.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.8249,
  

              'surface_temperature_celcius': 80.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -36.7089,
              'longitude': 143.3518,
              'surface_temperature_celcius': 93.0}},
 {'Hotspot': {'confidence': 97.0,
              'latitude': -36.601,
              'longitude': 144.6378,
              'surface_temperature_celcius': 80.0}},
 {'Hotspot': {'confidence': 95.0,
              'latitude': -36.8681,
              'longitude': 143.5186,
              'surface_temperature_celcius': 76.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.1999,
              'longitude': 143.8231,
              'surface_temperature_celcius': 90.0}},
 {'Hotspot': {'confidence': 91.0,
              'latitude': -37.6171,
              'longitude': 142.9426,
              'surface_temperature_celcius': 68.0}},
 {'Hotspot': {'confidence': 84.0,
              'latitude': -37.7587,
              'longitude': 142.8521,
              'surface_temperature_celc

 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.5043,
              'longitude': 146.3299,
              'surface_temperature_celcius': 93.0}},
 {'Hotspot': {'confidence': 93.0,
              'latitude': -36.1964,
              'longitude': 144.5217,
              'surface_temperature_celcius': 72.0}},
 {'Hotspot': {'confidence': 93.0,
              'latitude': -37.769,
              'longitude': 147.94,
              'surface_temperature_celcius': 72.0}},
 {'Hotspot': {'confidence': 90.0,
              'latitude': -36.1303,
              'longitude': 146.3606,
              'surface_temperature_celcius': 67.0}},
 {'Hotspot': {'confidence': 98.0,
              'latitude': -36.1345,
              'longitude': 146.351,
              'surface_temperature_celcius': 84.0}},
 {'Hotspot': {'confidence': 99.0,
              'latitude': -36.1586,
              'longitude': 145.6666,
              'surface_temperature_celcius': 85.0}},
 {'Hotspot': {'confidence': 96.0,
         

              'longitude': 149.32,
              'surface_temperature_celcius': 84.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.642,
              'longitude': 149.263,
              'surface_temperature_celcius': 65.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -37.634,
              'longitude': 149.237,
              'surface_temperature_celcius': 71.0}},
 {'Hotspot': {'confidence': 98.0,
              'latitude': -37.605,
              'longitude': 149.302,
              'surface_temperature_celcius': 83.0}},
 {'Hotspot': {'confidence': 99.0,
              'latitude': -37.6,
              'longitude': 149.325,
              'surface_temperature_celcius': 86.0}},
 {'Hotspot': {'confidence': 95.0,
              'latitude': -37.618,
              'longitude': 149.281,
              'surface_temperature_celcius': 76.0}},
 {'Hotspot': {'confidence': 100.0,
              'latitude': -36.282,
              'longitude': 146.157,
              's

In [10]:
# Task 2 Q(2)(C)
result = merged_hist.find( {
    "$or": [
        {'date':{ "$eq": '2021-12-15' } },
        {'date':{ "$eq": '2021-12-16' } } 
    ] },
    {'date':1, 'Hotspot.surface_temperature_celcius':1 , 'air_temperature_celcius':1 , 'relative_humidity' :1, 'max_wind_speed':1, '_id':0}
)

for x in result:
    pprint.pprint(x)

{'Hotspot': [{'surface_temperature_celcius': 42.0},
             {'surface_temperature_celcius': 36.0},
             {'surface_temperature_celcius': 38.0},
             {'surface_temperature_celcius': 40.0}],
 'air_temperature_celcius': 18.0,
 'date': '2021-12-15',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'Hotspot': [{'surface_temperature_celcius': 43.0},
             {'surface_temperature_celcius': 33.0},
             {'surface_temperature_celcius': 54.0},
             {'surface_temperature_celcius': 73.0},
             {'surface_temperature_celcius': 55.0},
             {'surface_temperature_celcius': 75.0},
             {'surface_temperature_celcius': 55.0},
             {'surface_temperature_celcius': 66.0},
             {'surface_temperature_celcius': 56.0},
             {'surface_temperature_celcius': 60.0},
             {'surface_temperature_celcius': 73.0},
             {'surface_temperature_celcius': 48.0},
             {'surface_temperature_celcius': 55.0},
     

In [11]:
# Task 2 Q(2)(D)

pprint.pprint(list(merged_hist.aggregate([
    { '$unwind' : '$Hotspot' },
    {'$match' :{ '$or' : [ { 'Hotspot.confidence' : { '$gte': 80}}, { 'Hotspot.confidence' : { '$lte': 100}}]} },
    {'$project' : {'_id':0, 'Hotspot.surface_temperature_celcius':1 , 'air_temperature_celcius':1, 'Hotspot.datetime':1, 'Hotspot.confidence' :1}}

     ])))


[{'Hotspot': {'confidence': 87.0,
              'datetime': '2021-03-06T05:06:30',
              'surface_temperature_celcius': 62.0},
  'air_temperature_celcius': 20.0},
 {'Hotspot': {'confidence': 85.0,
              'datetime': '2021-03-06T05:06:20',
              'surface_temperature_celcius': 59.0},
  'air_temperature_celcius': 20.0},
 {'Hotspot': {'confidence': 88.0,
              'datetime': '2021-03-07T04:16:10',
              'surface_temperature_celcius': 64.0},
  'air_temperature_celcius': 19.0},
 {'Hotspot': {'confidence': 68.0,
              'datetime': '2021-03-08T04:51:00',
              'surface_temperature_celcius': 55.0},
  'air_temperature_celcius': 21.0},
 {'Hotspot': {'confidence': 75.0,
              'datetime': '2021-03-08T00:30:30',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 21.0},
 {'Hotspot': {'confidence': 86.0,
              'datetime': '2021-03-09T13:23:40',
              'surface_temperature_celcius': 41.0},
  'air_tem

              'surface_temperature_celcius': 57.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 74.0,
              'datetime': '2021-03-26T04:47:20',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 74.0,
              'datetime': '2021-03-26T04:43:40',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 81.0,
              'datetime': '2021-03-26T04:43:20',
              'surface_temperature_celcius': 54.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 68.0,
              'datetime': '2021-03-26T04:41:10',
              'surface_temperature_celcius': 44.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 80.0,
              'datetime': '2021-03-26T04:40:50',
              'surface_temperature_celcius': 53.0},
  'air_temperature_celcius': 23.0},
 {'Hotspot': {'confidence': 75.0,
              'datetime

              'surface_temperature_celcius': 44.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 70.0,
              'datetime': '2021-03-31T04:57:40',
              'surface_temperature_celcius': 45.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 71.0,
              'datetime': '2021-03-31T04:57:40',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 76.0,
              'datetime': '2021-03-31T04:57:20',
              'surface_temperature_celcius': 50.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 86.0,
              'datetime': '2021-03-31T04:57:10',
              'surface_temperature_celcius': 61.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 79.0,
              'datetime': '2021-03-31T04:57:10',
              'surface_temperature_celcius': 52.0},
  'air_temperature_celcius': 14.0},
 {'Hotspot': {'confidence': 52.0,
              'datetime

  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 81.0,
              'datetime': '2021-04-03T03:51:00',
              'surface_temperature_celcius': 54.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 98.0,
              'datetime': '2021-04-03T03:51:00',
              'surface_temperature_celcius': 82.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 80.0,
              'datetime': '2021-04-03T03:51:00',
              'surface_temperature_celcius': 53.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 100.0,
              'datetime': '2021-04-03T03:51:00',
              'surface_temperature_celcius': 103.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 87.0,
              'datetime': '2021-04-03T03:50:50',
              'surface_temperature_celcius': 97.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 67.0,
              'datetime': '2021-04-03T03:50:50',
              'surface_t

  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 74.0,
              'datetime': '2021-04-04T04:32:40',
              'surface_temperature_celcius': 54.0},
  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 67.0,
              'datetime': '2021-04-04T04:32:40',
              'surface_temperature_celcius': 43.0},
  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 81.0,
              'datetime': '2021-04-04T04:32:40',
              'surface_temperature_celcius': 54.0},
  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 60.0,
              'datetime': '2021-04-04T04:32:40',
              'surface_temperature_celcius': 40.0},
  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 100.0,
              'datetime': '2021-04-04T04:32:40',
              'surface_temperature_celcius': 98.0},
  'air_temperature_celcius': 16.0},
 {'Hotspot': {'confidence': 72.0,
              'datetime': '2021-04-04T00:14:30',
              'surface_te

...

 {'Hotspot': {'confidence': 71.0,
              'datetime': '2021-07-29T04:07:50',
              'surface_temperature_celcius': 46.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 100.0,
              'datetime': '2021-07-31T13:21:00',
              'surface_temperature_celcius': 51.0},
  'air_temperature_celcius': 7.0},
 {'Hotspot': {'confidence': 92.0,
              'datetime': '2021-07-31T13:21:00',
              'surface_temperature_celcius': 43.0},
  'air_temperature_celcius': 7.0},
 {'Hotspot': {'confidence': 89.0,
              'datetime': '2021-08-01T00:20:30',
              'surface_temperature_celcius': 65.0},
  'air_temperature_celcius': 7.0},
 {'Hotspot': {'confidence': 78.0,
              'datetime': '2021-08-01T00:19:30',
              'surface_temperature_celcius': 51.0},
  'air_temperature_celcius': 7.0},
 {'Hotspot': {'confidence': 94.0,
              'datetime': '2021-08-02T03:45:40',
              'surface_temperature_celcius': 87.0},
  'air_temper

  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 83.0,
              'datetime': '2021-09-27T04:35:36',
              'surface_temperature_celcius': 56.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 86.0,
              'datetime': '2021-09-27T04:35:36',
              'surface_temperature_celcius': 60.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 72.0,
              'datetime': '2021-09-27T04:35:27',
              'surface_temperature_celcius': 45.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 65.0,
              'datetime': '2021-09-27T04:35:18',
              'surface_temperature_celcius': 39.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 79.0,
              'datetime': '2021-09-27T00:21:05',
              'surface_temperature_celcius': 49.0},
  'air_temperature_celcius': 15.0},
 {'Hotspot': {'confidence': 79.0,
              'datetime': '2021-09-27T00:21:05',
              'surface_tem

 {'Hotspot': {'confidence': 76.0,
              'datetime': '2021-10-10T04:04:19',
              'surface_temperature_celcius': 43.0},
  'air_temperature_celcius': 13.0},
 {'Hotspot': {'confidence': 100.0,
              'datetime': '2021-10-15T04:23:01',
              'surface_temperature_celcius': 87.0},
  'air_temperature_celcius': 13.0},
 {'Hotspot': {'confidence': 99.0,
              'datetime': '2021-10-15T04:23:01',
              'surface_temperature_celcius': 85.0},
  'air_temperature_celcius': 13.0},
 {'Hotspot': {'confidence': 78.0,
              'datetime': '2021-10-15T00:08:42',
              'surface_temperature_celcius': 46.0},
  'air_temperature_celcius': 13.0},
 {'Hotspot': {'confidence': 62.0,
              'datetime': '2021-10-16T23:56:43',
              'surface_temperature_celcius': 36.0},
  'air_temperature_celcius': 19.0},
 {'Hotspot': {'confidence': 94.0,
              'datetime': '2021-10-17T13:36:12',
              'surface_temperature_celcius': 43.0},
  'air_te

              'datetime': '2021-11-12T04:46:53',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 18.0},
 {'Hotspot': {'confidence': 67.0,
              'datetime': '2021-11-12T00:33:15',
              'surface_temperature_celcius': 54.0},
  'air_temperature_celcius': 18.0},
 {'Hotspot': {'confidence': 89.0,
              'datetime': '2021-11-12T00:33:15',
              'surface_temperature_celcius': 69.0},
  'air_temperature_celcius': 18.0},
 {'Hotspot': {'confidence': 87.0,
              'datetime': '2021-11-13T03:52:15',
              'surface_temperature_celcius': 61.0},
  'air_temperature_celcius': 24.0},
 {'Hotspot': {'confidence': 81.0,
              'datetime': '2021-11-13T03:52:14',
              'surface_temperature_celcius': 53.0},
  'air_temperature_celcius': 24.0},
 {'Hotspot': {'confidence': 64.0,
              'datetime': '2021-11-13T03:52:08',
              'surface_temperature_celcius': 42.0},
  'air_temperature_celcius': 24.0},
 {'Hotsp

              'surface_temperature_celcius': 86.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 69.0,
              'datetime': '2021-11-30T04:34:57',
              'surface_temperature_celcius': 48.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 60.0,
              'datetime': '2021-11-30T00:21:00',
              'surface_temperature_celcius': 42.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 83.0,
              'datetime': '2021-11-30T00:20:53',
              'surface_temperature_celcius': 58.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 87.0,
              'datetime': '2021-11-30T00:20:53',
              'surface_temperature_celcius': 63.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 95.0,
              'datetime': '2021-11-30T00:20:53',
              'surface_temperature_celcius': 76.0},
  'air_temperature_celcius': 26.0},
 {'Hotspot': {'confidence': 81.0,
              'datetime

In [12]:
# Task (2)(E)

pprint.pprint(list(merged_hist.aggregate([
        { '$unwind' : '$Hotspot' },
        { '$sort' : { 'Hotspot.surface_temperature_celcius' : -1}},
        { '$limit' : 10}
         ])))




[{'GHI_w/m2': 122.0,
  'Hotspot': {'confidence': 100.0,
              'datetime': '2021-04-18T04:52:00',
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124.0},
  '_id': ObjectId('62877ec8313b28f84f667b5e'),
  'air_temperature_celcius': 15.0,
  'date': '2021-04-18',
  'max_wind_speed': 9.9,
  'precipitation ': ' 0.00I',
  'precipitation_amount': 0.0,
  'precipitation_type': 'I',
  'relative_humidity': 56.1,
  'station': '948701',
  'windspeed_knots': 5.1},
 {'GHI_w/m2': 140.0,
  'Hotspot': {'confidence': 100.0,
              'datetime': '2021-04-04T04:32:50',
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123.0},
  '_id': ObjectId('62877ec8313b28f84f667b50'),
  'air_temperature_celcius': 16.0,
  'date': '2021-04-04',
  'max_wind_speed': 12.0,
  'precipitation ': ' 0.00I',
  'precipitation_amount': 0.0,
  'precipitation_type': 'I',
  'relative_humidi

In [13]:
# Task (2)(F)
pprint.pprint(list(merged_hist.aggregate([
    { '$unwind' : '$Hotspot'},
    { '$group' : { '_id' : '$date', 'count': {'$sum':1}}}
])))

[{'_id': '2021-01-06', 'count': 2},
 {'_id': '2021-12-11', 'count': 5},
 {'_id': '2021-01-20', 'count': 1},
 {'_id': '2021-03-30', 'count': 1},
 {'_id': '2021-08-05', 'count': 24},
 {'_id': '2021-10-21', 'count': 4},
 {'_id': '2021-07-27', 'count': 1},
 {'_id': '2021-12-23', 'count': 1},
 {'_id': '2021-10-28', 'count': 1},
 {'_id': '2021-12-13', 'count': 1},
 {'_id': '2021-05-21', 'count': 1},
 {'_id': '2021-08-25', 'count': 1},
 {'_id': '2021-07-15', 'count': 1},
 {'_id': '2021-07-22', 'count': 1},
 {'_id': '2021-09-09', 'count': 1},
 {'_id': '2021-08-31', 'count': 1},
 {'_id': '2021-10-08', 'count': 1},
 {'_id': '2021-09-13', 'count': 1},
 {'_id': '2021-01-17', 'count': 1},
 {'_id': '2021-03-02', 'count': 1},
 {'_id': '2021-10-05', 'count': 114},
 {'_id': '2021-02-18', 'count': 1},
 {'_id': '2021-03-16', 'count': 1},
 {'_id': '2021-03-14', 'count': 10},
 {'_id': '2021-08-19', 'count': 1},
 {'_id': '2021-05-09', 'count': 1},
 {'_id': '2021-11-15', 'count': 1},
 {'_id': '2021-01-14', '

In [14]:
# Task (2)(G)
result = merged_hist.find( 
    {'Hotspot.confidence' : { '$exists' : True ,"$lt": 70 }},
                            { 'Hotspot.$' : 1, '_id':0}
)

for x in result:
    pprint.pprint(x)

{'Hotspot': [{'confidence': 68.0,
              'datetime': '2021-03-08T04:51:00',
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55.0}]}
{'Hotspot': [{'confidence': 54.0,
              'datetime': '2021-03-09T03:57:00',
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44.0}]}
{'Hotspot': [{'confidence': 55.0,
              'datetime': '2021-03-10T04:43:00',
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42.0}]}
{'Hotspot': [{'confidence': 56.0,
              'datetime': '2021-03-13T23:58:50',
              'latitude': -37.0286,
              'longitude': 148.155,
              'surface_temperature_celcius': 42.0}]}
{'Hotspot': [{'confidence': 52.0,
              'datetime': '2021-03-15T00:42:50',
              'latitude': -37.4075,
              'longitude': 147.0233,
         

{'Hotspot': [{'confidence': 54.0,
              'datetime': '2021-08-13T05:13:30',
              'latitude': -34.848,
              'longitude': 141.5002,
              'surface_temperature_celcius': 41.0}]}
{'Hotspot': [{'confidence': 56.0,
              'datetime': '2021-08-14T04:15:40',
              'latitude': -37.4421,
              'longitude': 148.259,
              'surface_temperature_celcius': 39.0}]}
{'Hotspot': [{'confidence': 56.0,
              'datetime': '2021-09-23T23:50:43',
              'latitude': -37.559,
              'longitude': 148.037,
              'surface_temperature_celcius': 41.0}]}
{'Hotspot': [{'confidence': 59.0,
              'datetime': '2021-09-24T13:30:09',
              'latitude': -37.375,
              'longitude': 148.063,
              'surface_temperature_celcius': 35.0}]}
{'Hotspot': [{'confidence': 64.0,
              'datetime': '2021-09-26T03:52:14',
              'latitude': -37.087,
              'longitude': 145.37,
              'su

In [15]:
# Task(2)(H)
pprint.pprint(list(merged_hist.aggregate([
    { '$unwind' : '$Hotspot'},
    { '$match' : {'Hotspot.surface_temperature_celcius' : { '$exists' : True  }}},
    { '$group' : { '_id' : '$date', 'average_temperature': {'$avg':'$Hotspot.surface_temperature_celcius'}}}
])))

[{'_id': '2021-10-23', 'average_temperature': 38.0},
 {'_id': '2021-01-08', 'average_temperature': 58.0},
 {'_id': '2021-03-31', 'average_temperature': 48.72727272727273},
 {'_id': '2021-11-30', 'average_temperature': 52.41935483870968},
 {'_id': '2021-05-05', 'average_temperature': 51.70967741935484},
 {'_id': '2021-11-06', 'average_temperature': 41.5},
 {'_id': '2021-06-10', 'average_temperature': 44.0},
 {'_id': '2021-08-11', 'average_temperature': 45.5},
 {'_id': '2021-09-24', 'average_temperature': 53.57142857142857},
 {'_id': '2021-09-22', 'average_temperature': 45.0},
 {'_id': '2021-11-22', 'average_temperature': 61.5},
 {'_id': '2021-09-12', 'average_temperature': 58.25},
 {'_id': '2021-09-05', 'average_temperature': 42.46153846153846},
 {'_id': '2021-11-13', 'average_temperature': 47.0},
 {'_id': '2021-09-10', 'average_temperature': 44.0},
 {'_id': '2021-03-28', 'average_temperature': 60.925925925925924},
 {'_id': '2021-05-13', 'average_temperature': 58.611111111111114},
 {'_i

In [16]:
# Task (2)(I)

pprint.pprint(list(merged_hist.aggregate([
        { '$unwind' : '$Hotspot' },
        { '$sort' : { 'GHI_w/m2' : 1}},
        { '$limit' : 10}
         ])))

[{'GHI_w/m2': 47.0,
  'Hotspot': {'confidence': 54.0,
              'datetime': '2021-08-02T03:45:00',
              'latitude': -37.491,
              'longitude': 141.936,
              'surface_temperature_celcius': 40.0},
  '_id': ObjectId('62877ec8313b28f84f667bc8'),
  'air_temperature_celcius': 5.0,
  'date': '2021-02-08',
  'max_wind_speed': 5.1,
  'precipitation ': ' 0.00I',
  'precipitation_amount': 0.0,
  'precipitation_type': 'I',
  'relative_humidity': 38.6,
  'station': '948701',
  'windspeed_knots': 1.8},
 {'GHI_w/m2': 47.0,
  'Hotspot': {'confidence': 94.0,
              'datetime': '2021-08-02T03:45:40',
              'latitude': -37.4796,
              'longitude': 141.9403,
              'surface_temperature_celcius': 87.0},
  '_id': ObjectId('62877ec8313b28f84f667bc8'),
  'air_temperature_celcius': 5.0,
  'date': '2021-02-08',
  'max_wind_speed': 5.1,
  'precipitation ': ' 0.00I',
  'precipitation_amount': 0.0,
  'precipitation_type': 'I',
  'relative_humidity': 38.6

In [17]:
# Task(2)(J)
result = merged_hist.find( {
    "$and": [
        {'precipitation_type':{ "$eq": 'G' } },
        {'precipitation_amount':{ "$gte": 0.20 } },
        {'precipitation_amount':{ "$lte": 0.35 } } 
    ] },
)

for x in result:
    pprint.pprint(x)

{'GHI_w/m2': 157.0,
 'Hotspot': nan,
 '_id': ObjectId('62877ec8313b28f84f667aff'),
 'air_temperature_celcius': 19.0,
 'date': '2021-01-13',
 'max_wind_speed': 18.1,
 'precipitation ': ' 0.31G',
 'precipitation_amount': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 54.1,
 'station': '948700',
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146.0,
 'Hotspot': [{'confidence': 69.0,
              'datetime': '2021-03-29T00:48:40',
              'latitude': -34.2648,
              'longitude': 141.6325,
              'surface_temperature_celcius': 51.0}],
 '_id': ObjectId('62877ec8313b28f84f667b4a'),
 'air_temperature_celcius': 17.0,
 'date': '2021-03-29',
 'max_wind_speed': 21.0,
 'precipitation ': ' 0.24G',
 'precipitation_amount': 0.24,
 'precipitation_type': 'G',
 'relative_humidity': 49.9,
 'station': '948701',
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166.0,
 'Hotspot': [{'confidence': 84.0,
              'datetime': '2021-04-20T04:44:20',
              'latitude': -36.8871,
           